## Part 1: Data Loading and Cleaning

In [34]:
from google.colab import files
uploaded = files.upload()


Saving calgary_access_log.gz to calgary_access_log.gz


In [35]:
import gzip
import shutil

# Extract the contents of .gz into access_log
with gzip.open('calgary_access_log.gz', 'rb') as f_in:
    with open('access_log', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ Extracted file: access_log")


✅ Extracted file: access_log


In [36]:
import pandas as pd
import re
from datetime import datetime
from tqdm import tqdm

# Apache log format pattern
log_pattern = re.compile(
    r'(?P<remotehost>\S+) (?P<rfc931>\S+) (?P<authuser>\S+) '
    r'\[(?P<date>[^\]]+)\] "(?P<request>[^"]*)" '
    r'(?P<status>\d{3}) (?P<bytes>\S+)'
)

log_entries = []

with open('access_log', 'r', encoding='utf-8', errors='ignore') as file:
    for line in tqdm(file, desc="🔍 Parsing log entries"):
        match = log_pattern.match(line)
        if match:
            entry = match.groupdict()

            # Parse datetime
            try:
                entry['datetime'] = datetime.strptime(entry['date'], "%d/%b/%Y:%H:%M:%S %z")
            except:
                continue  # skip malformed dates

            # Parse request
            parts = entry['request'].split()
            if len(parts) == 3:
                entry['method'], entry['filename'], entry['protocol'] = parts
            else:
                entry['method'], entry['filename'], entry['protocol'] = None, None, None

            # Convert bytes
            entry['bytes'] = int(entry['bytes']) if entry['bytes'].isdigit() else None

            # Convert status
            try:
                entry['status'] = int(entry['status'])
            except:
                entry['status'] = None

            # Extract file extension
            if entry['filename'] and '.' in entry['filename']:
                entry['extension'] = entry['filename'].split('.')[-1].lower()
            else:
                entry['extension'] = None

            log_entries.append(entry)


🔍 Parsing log entries: 726739it [00:14, 50814.65it/s]


In [37]:
# Convert to DataFrame
df = pd.DataFrame(log_entries)

print(f"\n✅ Successfully parsed {len(df)} valid log entries.")
df.head()



✅ Successfully parsed 724836 valid log entries.


,remotehost,rfc931,authuser,date,request,status,bytes,datetime,method,filename,protocol,extension
0,local,-,-,24/Oct/1994:13:41:41 -0600,GET index.html HTTP/1.0,200,150.0,1994-10-24 13:41:41-06:00,GET,index.html,HTTP/1.0,html
1,local,-,-,24/Oct/1994:13:41:41 -0600,GET 1.gif HTTP/1.0,200,1210.0,1994-10-24 13:41:41-06:00,GET,1.gif,HTTP/1.0,gif
2,local,-,-,24/Oct/1994:13:43:13 -0600,GET index.html HTTP/1.0,200,3185.0,1994-10-24 13:43:13-06:00,GET,index.html,HTTP/1.0,html
3,local,-,-,24/Oct/1994:13:43:14 -0600,GET 2.gif HTTP/1.0,200,2555.0,1994-10-24 13:43:14-06:00,GET,2.gif,HTTP/1.0,gif
4,local,-,-,24/Oct/1994:13:43:15 -0600,GET 3.gif HTTP/1.0,200,36403.0,1994-10-24 13:43:15-06:00,GET,3.gif,HTTP/1.0,gif


## Part 2: Analysis Questions

In [38]:
# Q1: Count of total log records
print("Q1: Total number of HTTP requests (log entries):")
total_logs = len(df)
print(total_logs)

Q1: Total number of HTTP requests (log entries):
724836


In [39]:
# Q2: Count of unique hosts
print("\nQ2: Number of unique remote hosts:")
unique_hosts = df['remotehost'].nunique()
print(unique_hosts)


Q2: Number of unique remote hosts:
2


In [40]:
# Q3: Date-wise unique filename counts
print("\nQ3: Unique filenames per date (format: dd-MMM-yyyy):")

# Ensure the 'datetime' column is of datetime type, coercing errors
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Drop rows where 'datetime' is NaT (Not a Time), which indicates a failed conversion
df_cleaned = df.dropna(subset=['datetime']).copy() # Create a copy to avoid SettingWithCopyWarning

# Now use the cleaned DataFrame for the rest of the calculation
df_cleaned['date_str'] = df_cleaned['datetime'].dt.strftime('%d-%b-%Y')
unique_filenames_per_day = df_cleaned.groupby('date_str')['filename'].nunique()
print(unique_filenames_per_day.to_dict())


Q3: Unique filenames per date (format: dd-MMM-yyyy):
{'01-Aug-1995': 669, '01-Jul-1995': 387, '01-Jun-1995': 590, '01-May-1995': 467, '01-Oct-1995': 552, '01-Sep-1995': 328, '02-Apr-1995': 438, '02-Aug-1995': 855, '02-Jul-1995': 397, '02-Jun-1995': 513, '02-May-1995': 701, '02-Oct-1995': 871, '02-Sep-1995': 349, '03-Apr-1995': 795, '03-Aug-1995': 582, '03-Jul-1995': 433, '03-Jun-1995': 398, '03-May-1995': 589, '03-Oct-1995': 846, '03-Sep-1995': 212, '04-Apr-1995': 821, '04-Aug-1995': 715, '04-Jul-1995': 610, '04-Jun-1995': 353, '04-May-1995': 684, '04-Oct-1995': 889, '04-Sep-1995': 340, '05-Apr-1995': 891, '05-Aug-1995': 507, '05-Jul-1995': 607, '05-Jun-1995': 494, '05-May-1995': 608, '05-Oct-1995': 846, '05-Sep-1995': 411, '06-Apr-1995': 678, '06-Aug-1995': 448, '06-Jul-1995': 522, '06-Jun-1995': 662, '06-May-1995': 517, '06-Oct-1995': 868, '06-Sep-1995': 549, '07-Apr-1995': 776, '07-Aug-1995': 608, '07-Jul-1995': 428, '07-Jun-1995': 485, '07-May-1995': 725, '07-Oct-1995': 468, '07-S

In [41]:
# Q4: Number of 404 response codes
print("\nQ4: Total number of 404 errors:")
total_404 = df[df['status'] == 404].shape[0]
print(total_404)


Q4: Total number of 404 errors:
23517


In [42]:
# Q5: Top 15 filenames with 404 responses
print("\nQ5: Top 15 filenames causing 404 errors:")
top_404_files = df[df['status'] == 404]['filename'].value_counts().head(15)
print(list(top_404_files.items()))


Q5: Top 15 filenames causing 404 errors:
[('index.html', 4694), ('4115.html', 902), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 258), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('40.html', 148), ('3414.gif', 148), ('488.gif', 148), ('9678.gif', 142)]


In [43]:
# Q6: Top 15 file extensions that caused 404 errors
print("\nQ6: Top 15 file extensions causing 404 errors:")
top_404_ext = df[df['status'] == 404]['extension'].value_counts().head(15)
print(list(top_404_ext.items()))


Q6: Top 15 file extensions causing 404 errors:
[('html', 12145), ('gif', 7337), ('xbm', 824), ('ps', 754), ('jpg', 531), ('txt', 508), ('htm', 108), ('cgi', 77), ('com', 45), ('z', 41), ('dvi', 40), ('com/', 37), ('ca', 36), ('hmtl', 30), ('util', 29)]


In [44]:
# Q7: Total bandwidth transferred per day for July 1995
print("\nQ7: Bandwidth transferred per day in July 1995:")
july_df = df[(df['datetime'].dt.month == 7) & (df['datetime'].dt.year == 1995)]
july_bandwidth = july_df.dropna(subset=['bytes']).groupby(
    july_df['datetime'].dt.strftime('%d-%b-%Y')
)['bytes'].sum()
print(july_bandwidth.astype(int).to_dict())


Q7: Bandwidth transferred per day in July 1995:
{'01-Jul-1995': 11349799, '02-Jul-1995': 8656918, '03-Jul-1995': 13596612, '04-Jul-1995': 26573988, '05-Jul-1995': 19541225, '06-Jul-1995': 19755015, '07-Jul-1995': 9427822, '08-Jul-1995': 5403491, '09-Jul-1995': 4660556, '10-Jul-1995': 14917754, '11-Jul-1995': 22507207, '12-Jul-1995': 17367065, '13-Jul-1995': 15989234, '14-Jul-1995': 19186430, '15-Jul-1995': 15773233, '16-Jul-1995': 9016378, '17-Jul-1995': 19601338, '18-Jul-1995': 17099761, '19-Jul-1995': 17851725, '20-Jul-1995': 20752623, '21-Jul-1995': 25491617, '22-Jul-1995': 8136259, '23-Jul-1995': 9593870, '24-Jul-1995': 22308265, '25-Jul-1995': 24561635, '26-Jul-1995': 24995540, '27-Jul-1995': 25969995, '28-Jul-1995': 36460693, '29-Jul-1995': 11700624, '30-Jul-1995': 23189598, '31-Jul-1995': 30730715}


In [45]:
# Q8: Hourly request distribution (0–23)
print("\nQ8: Number of HTTP requests per hour (0–23):")
df['hour'] = df['datetime'].dt.hour
hourly_dist = df['hour'].value_counts().sort_index()
print(hourly_dist.to_dict())


Q8: Number of HTTP requests per hour (0–23):
{0.0: 11598, 1.0: 9913, 2.0: 9403, 3.0: 8147, 4.0: 7820, 5.0: 8283, 6.0: 9798, 7.0: 11930, 8.0: 17351, 9.0: 21681, 10.0: 25713, 11.0: 28665, 12.0: 26845, 13.0: 30089, 14.0: 29792, 15.0: 28149, 16.0: 28286, 17.0: 23312, 18.0: 17862, 19.0: 17325, 20.0: 17488, 21.0: 15965, 22.0: 14587, 23.0: 13613}


In [46]:
# Q9: Top 10 most frequently requested filenames
print("\nQ9: Top 10 most requested filenames:")
top_requested = df['filename'].value_counts().head(10)
print(list(top_requested.items()))



Q9: Top 10 most requested filenames:
[('index.html', 139528), ('3.gif', 24006), ('2.gif', 23595), ('4.gif', 8018), ('244.gif', 5148), ('5.html', 5010), ('4097.gif', 4874), ('8870.jpg', 4492), ('6733.gif', 4278), ('8472.gif', 3843)]


In [47]:
# Q10: Count of each HTTP response status code
print("\nQ10: Distribution of HTTP response status codes:")
status_distribution = df['status'].value_counts().sort_index()
print(status_distribution.to_dict())


Q10: Distribution of HTTP response status codes:
{200: 568345, 302: 30295, 304: 97792, 400: 15, 401: 46, 403: 4741, 404: 23517, 500: 42, 501: 43}
